In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [8]:
data=pd.read_csv("D:\ML\ML Assignment\Global_Pollution_Analysis.csv")
df=pd.DataFrame(data)
print(df.head())
print(df.columns)

        Country  Year  Air_Pollution_Index  Water_Pollution_Index  \
0       Hungary  2005               272.70                 124.27   
1     Singapore  2001                86.72                  60.34   
2       Romania  2016                91.59                  83.36   
3  Cook Islands  2018               280.61                  67.16   
4      Djibouti  2008               179.16                 127.53   

   Soil_Pollution_Index  Industrial_Waste (in tons)  \
0                 51.95                    94802.83   
1                117.22                    56283.92   
2                121.72                    56256.02   
3                 93.58                    74864.73   
4                121.55                    76862.06   

   Energy_Rec   overed (in GWh)  CO2_Emissions (in MT)  Renewable_Energy (%)  \
0                        158.14                   5.30                 41.11   
1                        498.04                   6.34                 36.44   
2             

In [9]:
#Handle Missing Values
print(df.isnull().sum())
#checking for duplicated values
print("Number of duplicate rows:", df.duplicated().sum())


Country                                   0
Year                                      0
Air_Pollution_Index                       0
Water_Pollution_Index                     0
Soil_Pollution_Index                      0
Industrial_Waste (in tons)                0
Energy_Rec   overed (in GWh)              0
CO2_Emissions (in MT)                     0
Renewable_Energy (%)                      0
Plastic_Waste_Produced (in tons)          0
Energy_Consumption_Per_Capita (in MWh)    0
Population (in millions)                  0
GDP_Per_Capita (in USD)                   0
dtype: int64
Number of duplicate rows: 0


In [10]:
#normalize 
print(df.columns)
withoutscaled_data=df[["Air_Pollution_Index","Water_Pollution_Index","Soil_Pollution_Index"]]
scaled=StandardScaler()
scaled_df=scaled.fit_transform(withoutscaled_data)



Index(['Country', 'Year', 'Air_Pollution_Index', 'Water_Pollution_Index',
       'Soil_Pollution_Index', 'Industrial_Waste (in tons)',
       'Energy_Rec   overed (in GWh)', 'CO2_Emissions (in MT)',
       'Renewable_Energy (%)', 'Plastic_Waste_Produced (in tons)',
       'Energy_Consumption_Per_Capita (in MWh)', 'Population (in millions)',
       'GDP_Per_Capita (in USD)'],
      dtype='object')


In [11]:
#encoded
df_encoded=pd.get_dummies(
    df,
    columns=["Country","Year"],
    drop_first=True,
    dtype=int
)
print(df_encoded.columns)

Index(['Air_Pollution_Index', 'Water_Pollution_Index', 'Soil_Pollution_Index',
       'Industrial_Waste (in tons)', 'Energy_Rec   overed (in GWh)',
       'CO2_Emissions (in MT)', 'Renewable_Energy (%)',
       'Plastic_Waste_Produced (in tons)',
       'Energy_Consumption_Per_Capita (in MWh)', 'Population (in millions)',
       ...
       'Year_2010', 'Year_2011', 'Year_2012', 'Year_2013', 'Year_2014',
       'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019'],
      dtype='object', length=204)


In [12]:
df['Energy_Consumption_Per_Capita (in MWh)'].describe()
df["pollution_level"]=(df["Air_Pollution_Index"]+df["Soil_Pollution_Index"]+df["Water_Pollution_Index"])

low=df["pollution_level"].quantile(0.33)
high=df["pollution_level"].quantile(0.66)
df["pollution_Severity"]=pd.cut(
    df["pollution_level"],
    bins=[0,low,high,float("inf")],
    labels=[0,1,2]
)
